In [1]:
import lseg.data as ld
import pandas as pd
from lseg.data.content import news
from IPython.display import HTML

In [2]:
from langchain_ollama.chat_models import ChatOllama

# ChatNVIDIA.get_available_models()
llm = ChatOllama(model="llama3.1:8b-instruct-q8_0", temperature=0,)


In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x125d4f010 {name='workspace'}>

In [4]:
definition = news.top_news.hierarchy.Definition()
response = definition.get_data()
energy_top_news_id = response.data.hierarchy['Commodities']['Energy'].top_news_id
definition = news.top_news.Definition(energy_top_news_id)
news_response = definition.get_data()
print(news_response.data.df['storyId'].iloc(0)[0])

urn:newsml:reuters.com:20250930:nL2N3VH0HA


In [5]:
#Retrieve the first top news story
storyId = news_response.data.df["storyId"].iloc(0)[0]
news_story = ld.news.get_story(storyId)
with open('news_story.html', 'w+') as f:
    f.write(news_story)
    
display(HTML(news_story))

In [6]:
from langchain_community.document_loaders import BSHTMLLoader

loader = BSHTMLLoader('news_story.html')
docs = loader.load()
docs

[Document(metadata={'source': 'news_story.html', 'title': ''}, page_content="OPEC+ may accelerate hikes after adding 137,000 bpd in OctNovember hike could be 2 or 3 times the size of October hikeRussia uncomfortable with large hike, source saysEight OPEC+ countries meet on SundayAdds more sources, OPEC statement on XBy Alex Lawler, Olesya Astakhova and Dmitry ZhdannikovLONDON, Sept 30 (Reuters) - OPEC+ may speed up production increases in November from the 137,000 barrels per day hike it made for October at its meeting on Sunday as its leader Saudi Arabia pushes to regain market share, three sources familiar with the talks said.The group has made no final decision yet and member Russia could oppose a larger increase because it is unable to raise output owing to Western sanctions and is worried about weakening seasonal demand, one of the three sources said.Eight members of OPEC+ could agree to raise production in November by 274,000-411,000 bpd, or two or three times higher than the Oct

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template("""Write a concise summary of the following: '{context}' and tell me how it will affect Shell price""")

# Instantiate chain
chain = prompt | llm

# Invoke chain
result = chain.invoke({"context": docs})
print(result)

content="Here is a concise summary of the article:\n\nOPEC+ (Organization of the Petroleum Exporting Countries and its allies) may increase oil production by 274,000-411,000 barrels per day (bpd) in November, which is two to three times higher than the October increase. This decision will be made at an online meeting on Sunday. Russia may oppose a larger increase due to Western sanctions limiting its output and concerns about weakening seasonal demand.\n\nAs for how this will affect Shell prices, it's difficult to predict with certainty. However, if OPEC+ does decide to increase production by 274,000-411,000 bpd, it could lead to:\n\n* Increased oil supply in the market\n* Potential downward pressure on oil prices\n* Reduced demand for oil from refineries like Shell, which may lead to lower refining margins and potentially lower gasoline prices at the pump.\n\nHowever, if OPEC+ decides to increase production by 500,000 bpd or more, as some sources suggest, it could lead to:\n\n* A sign

In [11]:
print(result.content)

Here is a concise summary of the article:

OPEC+ (Organization of the Petroleum Exporting Countries and its allies) may increase oil production by 274,000-411,000 barrels per day (bpd) in November, which is two to three times higher than the October increase. This decision will be made at an online meeting on Sunday. Russia may oppose a larger increase due to Western sanctions limiting its output and concerns about weakening seasonal demand.

As for how this will affect Shell prices, it's difficult to predict with certainty. However, if OPEC+ does decide to increase production by 274,000-411,000 bpd, it could lead to:

* Increased oil supply in the market
* Potential downward pressure on oil prices
* Reduced demand for oil from refineries like Shell, which may lead to lower refining margins and potentially lower gasoline prices at the pump.

However, if OPEC+ decides to increase production by 500,000 bpd or more, as some sources suggest, it could lead to:

* A significant increase in o